<a href="https://colab.research.google.com/github/ameek2/CISC662/blob/master/tensorflowBenchmarkRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.python.framework.ops as ops
import os
import tensorflow_datasets as tfds

!pip install keras-flops


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Imports. Will cull as needed

Basic code stolen from
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb#scrollTo=3wF5wszaj97Y

In [2]:
g = tf.Graph()
with g.as_default():
  mnist = tf.keras.datasets.mnist

  (x_train, y_train), (x_test, y_test) = mnist.load_data()
  x_train, x_test = x_train/ 255.0, x_test/255.0
  model = tf.keras.models.Seqeuential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
  ])

  predictions = model(x_train[:1]).numpy()

  tf.nn.softmax(predictions).numpy()
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  loss_fn(y_train[:1], predictions).numpy()

  model.compile(optimizer='adam',loss=loss_fn, metrics= ['accuracy'])
  model.fit(x_train,y_train,epochs=5)
  model.evaluate(x_test, y_test, verbose=2)
  probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
  probability_model(x_test[:5])

for op in g.get_operations():
  flops = ops.get_stats_for_node_def(g, op.node_def, 'flops').value
  if flops is not None:
    print ('TF stats gives', flops)

11490434/11490434 [==============================] - 0s 0us/step


AttributeError: ignored

resnet18

resnet34

resnet50

resnet101

resnet152

vgg11

vgg13

vgg16

vgg19

wide_resnet50_2

alexnet

mobilenet_v2

efficientnet_b0 

above is the list of models tested in pytorch. below is the intersecting models available built-in from keras.applications: https://keras.io/api/applications/

code uses the library keras-flops from tokusumi on github: https://github.com/tokusumi/keras-flops

In [23]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from keras_flops import get_flops

# build model
inp = Input((32, 32, 3))
x = Conv2D(32, kernel_size=(3, 3), activation="relu")(inp)
x = Conv2D(64, (3, 3), activation="relu")(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
out = Dense(10, activation="softmax")(x)
model = Model(inp, out)

from keras.applications import ResNet50
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import MobileNetV2
from keras.applications import EfficientNetB0


# Calculae FLOPS
flops = get_flops(model, batch_size=1)
#print(f"FLOPS: {flops / 10 ** 9:.03} G")
#print(flops)
# >>> FLOPS: 0.0338 G

model = ResNet50(weights="imagenet")
print("Resnet50, flops: ", get_flops(model, batch_size=1))

model = VGG16(weights="imagenet", include_top=False)
print("VGG16, flops: ", get_flops(model, batch_size=1))

model = VGG19(weights="imagenet")
print("VGG19, flops: ", get_flops(model, batch_size=1))

model = MobileNetV2()
print("MobileNetV2, flops: ", get_flops(model, batch_size=1))

model = EfficientNetB0()
print("EfficientNetB0, flops: ", get_flops(model, batch_size=1))


Resnet50, flops:  7755299312
VGG16, flops:  0
VGG19, flops:  39285112688
14536120/14536120 [==============================] - 0s 0us/step
MobileNetV2, flops:  615292208
21834768/21834768 [==============================] - 0s 0us/step
EfficientNetB0, flops:  803200647
